In [ ]:
import os

# --- Add these lines at the very top of your script ---
# This must be done BEFORE importing numpy or other scientific libraries.
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['VECLIB_MAXIMUM_THREADS'] = '1'
os.environ['NUMEXPR_NUM_THREADS'] = '1'

import numpy as np
import multiprocessing
import matplotlib.pyplot as plt
from collections import defaultdict
from itertools import product
import time
from device import Device
from NEGF_sim_git.src.archive.rgf import GreensFunction
from hamiltonian import Hamiltonian
from device import Device
from NEGF_sim_git.src.archive.rgf import GreensFunction
import scipy as sp
from hamiltonian import Hamiltonian
from helper import Helper_functions
import scipy.sparse as spa
import numpy as np
import scipy.sparse as sp
from lead_self_energy import LeadSelfEnergy
from scipy.sparse import bmat, identity, random, csc_matrix
from scipy.sparse.linalg import eigsh, eigs, spsolve
import time
from charge import Charge

In [2]:
dev = Device(2e-9, 1e-9)
ham = Hamiltonian(dev)
lse = LeadSelfEnergy(dev, ham)

sl = lse.self_energy("left", 0,0)
print(np.trace(sl))
sr = lse.self_energy("right", 0,0)
print(np.trace(sr))

(7.991148054905903-0.00010475260124397965j)
(7.991148054905855-0.00010475260124397984j)


In [3]:
charge = Charge(dev)

charge._calculate_dos_point(0,0)

/home/nandan_diwan/Desktop/NEGF_sim/.venv/lib/python3.11/site-packages/scipy/sparse/_index.py:210: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil and dok are more efficient.
  self._set_arrayXarray(i, j, x)


0.00010151758248318354

In [7]:
charge._calculate_dos_point(-1,0)

1.2753421344592795